# Tutorial 08 - Character Prediction with LSTMs

This tutorial is based on the following two blog posts:
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://towardsdatascience.com/writing-like-shakespeare-with-machine-learning-in-pytorch-d77f851d910c


The goal of this tutorial is to build a character level RNN and train it on a text corpus of some of Shakespeare's work. We will then use the trained RNN to hallucinate new text (one character at a time). 

![caption](charseq.jpeg)

## Setup and Data Loading

We will use a small amount of Shakespears work to train the character prediction LSTM. Feel free to experiment with your own text data of choice!

In [1]:
# Importing libraries
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Open the text file and show the first 100 characters. 

In [2]:
# Open shakespeare text file and read in data as `text`
with open('shakespeare.txt', 'r') as f:
    text = f.read()
    
# Showing the first 100 characters
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

## Data Preparation

We need to encode the characters in some way. We will simply map each unique character in the text to an integer (and vice versa).

In [3]:
# encoding the text and map each character to an integer and vice versa

# We create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# Encode the text
encoded = np.array([char2int[ch] for ch in text])

# Showing the first 100 encoded characters
encoded[:100]

array([10, 63, 31, 21, 23, 36, 14, 63, 23, 63, 62, 17, 49, 56, 47,  5, 17,
       53,  1, 31, 17, 36, 24, 17, 36,  0, 31,  1, 46, 17, 17, 48, 36, 55,
       49, 28, 36, 53, 38, 31, 23, 51, 17, 31, 19, 36, 51, 17, 55, 31, 36,
       30, 17, 36, 21,  0, 17, 55, 26, 13, 47, 47, 27, 11, 11, 56, 47, 22,
        0, 17, 55, 26, 19, 36, 21,  0, 17, 55, 26, 13, 47, 47, 10, 63, 31,
       21, 23, 36, 14, 63, 23, 63, 62, 17, 49, 56, 47, 42,  1, 38])

Let's also define some helper functions that map the ineger IDs to one-hot labels. The one-hot encodings will be used as inputs and targets for the network training (similar to multi-class classification).

In [4]:
# Defining method to encode one hot labels
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

Finally, define a helper function that will create mini-batches for training. 

In [5]:
# Defining method to make mini-batches for training
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

## Define the Recurrent Neural Network Model

First check if a GPU is available for training

In [6]:
# Check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


Now, it’s time to define the RNN network. We’re going to implement dropout for regularization and also create character dictionaries within the network. We’ll have LSTM units followed by a fully connected layer.

We’ll name it Char-RNN because rather than having the input sequence be words, we’re going to look at the individual letters/characters instead.

In the forward function, we’ll propagate the input and hidden state values through the LSTM layer to get the output and next hidden state values. After performing dropout, we reshape the output value to make it the proper dimensions for the fully connected layer.

Finally, we also have a section in the RNN for initializing the hidden state values for the correct batch size if using mini-batches.

In [7]:
# Declaring the model
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        #define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Training Code

Time for training! We declare a function, where we define an optimizer (Adam) and loss (cross entropy). We then create the training and validation data and initialize the hidden state of the RNN. 
We loop over the training set, each time encoding the data into one-hot vectors, performing forward and backpropagation, and updating the network parameters.

Every once a while, we generate some loss statistics (training loss and validation loss) to let us know if the model is training correctly.

In [8]:
# Declaring the train method
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

Now, we just declare the hyperparameters for our model, create an instance for it, and train it!

In [9]:
# Define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

CharRNN(
  (lstm): LSTM(65, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=65, bias=True)
)
Epoch: 1/20... Step: 50... Loss: 3.3330... Val Loss: 3.3380
Epoch: 2/20... Step: 100... Loss: 3.2694... Val Loss: 3.2729
Epoch: 2/20... Step: 150... Loss: 2.9064... Val Loss: 2.8759
Epoch: 3/20... Step: 200... Loss: 2.5984... Val Loss: 2.5299
Epoch: 4/20... Step: 250... Loss: 2.4060... Val Loss: 2.3536
Epoch: 4/20... Step: 300... Loss: 2.2965... Val Loss: 2.2341
Epoch: 5/20... Step: 350... Loss: 2.1986... Val Loss: 2.1361
Epoch: 6/20... Step: 400... Loss: 2.1243... Val Loss: 2.0637
Epoch: 6/20... Step: 450... Loss: 2.0567... Val Loss: 2.0118
Epoch: 7/20... Step: 500... Loss: 1.9926... Val Loss: 1.9688
Epoch: 8/20... Step: 550... Loss: 1.9433... Val Loss: 1.9222
Epoch: 8/20... Step: 600... Loss: 1.8732... Val Loss: 1.8865
Epoch: 9/20... Step: 650... Loss: 1.8484... Val Loss: 1.8548
Epoch: 9/20... Step: 700... Loss: 1.7971

## Generating new Text

After training, we create a method to predict the next character from the trained RNN with forward propagation.

In [10]:
# Defining a method to generate the next character
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

Then, we define a sampling method that will use the previous method to generate an entire string of text, first using the characters in the first word and then using a loop to generate the next words using the top_k function, which chooses the letter with the highest probability to be next.

In [11]:
# Declaring a method to generate new text
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

Finally, just call the method, define the size you want (e.g., 1000 characters) and the starting character (e.g., ‘A’) and get the result:

In [12]:
# Generating new text
print(sample(net, 1000, prime='A', top_k=5))

Ad, what, the person, thou hast true his fall,
When he desires the persuned take and seads
The mightess still with her to have be a serpent;
These the most charge and himself, that I well be not
Wompone well sight, trumpet talk to her their son
The bods is that and men that stone the consures it
Than have to bear treaton.

CLARENCE:
Thou day'st the since weeping to be a crief
And will not teeth.'
The steads some state, thou a stale storn with the
send, there thither to his seeth was not all at time,
And will see this too such a man; and steal a too
to be he with a subblath of the struke of the
forselves, and thy clooks and such, which was not bard,
The way, a man of this some senter should have dead.

KING EDWARD IV:
Take a child bring of myself, thy doob.

Second Minderer:
And I will be make a sous of your highness
Is so meaning, by my foult of you.

Second Comen:
And strike, thy brother with him.

SAMPLOR:
Why, so? thou wellst as all, and when you will be
That is the cried.

SICINIUS